## input:    länkar till lopp
## scrape:   utdelning från varje v75 omg (inkl datum, bana)
## output:   utdelning.csv

In [1]:
#!apt install chromium-chromedriver
!pip install selenium


import pandas as  pd 
import numpy as np
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

import logging
logging.basicConfig(filename='bet.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)


In [2]:

def getBana(the_driver, avd, after=False):
    hack = '2' if avd == '7' else '1'     # när jag kör direkt efter omgångens slut
    #hack = '1' if avd == '7' else '1'    # när jag kör innan?

    #print('bana avd', avd, 'after', after, 'hack', hack)
    end_string = '/div['+hack+']/div/div[1]/div/div[2]/div[2]/div[1]/span'
    if after:
        end_string = '/div['+hack + \
            ']/div[1]/div[1]/div/div[2]/div[2]/div[1]/span'

    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div[' + \
        avd+']/div/div' + end_string

    bana = the_driver.find_element_by_xpath(pth)

    return bana.text

### utdelning ###
def getUtd(the_driver):
    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[5]/div[3]/ol/li[1]/span[3]'
    utd7= the_driver.find_element_by_xpath(pth).text
    utd7=utd7.replace(' ','')
    utd7=utd7.replace('kr','')
    utd7=utd7.replace('Jackpot','0')
    
    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[5]/div[3]/ol/li[2]/span[3]'
    utd6= the_driver.find_element_by_xpath(pth).text
    utd6=utd6.replace(' ','')
    utd6=utd6.replace('kr','')
    utd6=utd6.replace('Jackpot','0')
    
    pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[5]/div[3]/ol/li[3]/span[3]'
    utd5= the_driver.find_element_by_xpath(pth).text
    utd5=utd5.replace(' ','')
    utd5=utd5.replace('kr','')
    utd5=utd5.replace('Jackpot','0')
    
    return int(utd7),int(utd6),int(utd5)



In [3]:
def init_scraping(url):
    print('init')
    # get the webdriver
    driver_r = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')  
    print(f"get v75 omg 1")
    driver_r.get(url)
    driver_r.implicitly_wait(30) # seconds
    print('fullscreen')
    driver_r.fullscreen_window()

    import os
    input('klicka bort popup och tryck enter här')
    
    return driver_r

In [4]:
def do_scraping(url,driver_r):
    driver_r.get(url)
    print('efter get url')
    driver_r.implicitly_wait(2) # seconds
    #driver_r.fullscreen_window()
    driver_r.implicitly_wait(5) # seconds
    
    dat = url.split('spel/')[1][0:10]  #gräv ut datum  
    print(dat)
    bana = getBana(driver_r,str(1))
    utd7,utd6,utd5 = getUtd(driver_r)
    return [dat,bana,utd7,utd6,utd5]


In [5]:
utdData=[]
omg = pd.read_csv('vissa_lopp.csv',header=None)
omg.columns=['url']
print(f'{omg.size} omgångar')
driver_r = init_scraping(omg.iloc[0].url+'resultat')

for url in omg.url:
    #print(url)
    utdData.append(do_scraping(url+'resultat',driver_r))
    
utdelning = pd.DataFrame()

utdelning = pd.concat([utdelning,pd.DataFrame(utdData)])

#df.columns=['datum','bana','avd','nr','häst','streck']
utdelning.columns=['datum','bana','7rätt','6rätt','5rätt']
print(utdelning.info())
print(utdelning.datum.unique().size )


EmptyDataError: No columns to parse from file

In [ ]:
utdelning.to_csv('utdelning.csv')

In [ ]:
driver_r.quit()

In [ ]:
utdelning['5rätt'] = utdelning['5rätt'].astype(int)
#utdelning['7rätt'].astype(int)
utdelning.info()